# Prueba Técnica Mercado Libre - Jose Currea

In [75]:
import pandas as pd
from pandas import json_normalize
import numpy as np
pd.set_option('display.max_columns', None)  # Muestra todas las columnas
pd.set_option('display.max_rows', None)  # Muestra todas las filas

In [76]:
#API Categorias
import requests
cats = requests.get("https://api.mercadolibre.com/sites/MCO/categories")
cats_json = cats.json()
df_cats = pd.DataFrame.from_dict(cats_json)
df_cats.head(50)

,id,name
0,MCO1747,Accesorios para Vehículos
1,MCO441917,Agro
2,MCO1403,Alimentos y Bebidas
3,MCO1071,Animales y Mascotas
4,MCO1367,Antigüedades y Colecciones
5,MCO1368,"Arte, Papelería y Mercería"
6,MCO1384,Bebés
7,MCO1246,Belleza y Cuidado Personal
8,MCO40433,Boletas para Espectáculos
9,MCO1039,Cámaras y Accesorios


In [77]:
# Api items
cat_id = 'MCO1276'
total_results = 1000
offs = 10

# DF para los items
df_items = pd.DataFrame()

# Iterando sobre los offsets
for offset in range(0, total_results, offs):
    url = f'https://api.mercadolibre.com/sites/MCO/search?category={cat_id}&offset={offset}'
    request = requests.get(url)
    items = request.json()
    df_items = pd.concat([df_items, pd.DataFrame(items['results'])], ignore_index=True)

#Modificacion columnas
df_items['item_id']=df_items['id']
df_items = df_items.drop('id', axis=1)


In [78]:
df_items.head()

,title,condition,thumbnail_id,catalog_product_id,listing_type_id,permalink,buying_mode,site_id,category_id,domain_id,thumbnail,currency_id,order_backend,price,original_price,sale_price,available_quantity,official_store_id,use_thumbnail_id,accepts_mercadopago,shipping,stop_time,seller,attributes,installments,winner_item_id,catalog_listing,discounts,promotions,inventory_id,official_store_name,differential_pricing,variation_filters,variations_data,item_id
0,Pastillas Shimano B05s Originales 50% Más Resi...,new,831848-MLU71764372657_092023,MCO27114730,gold_special,https://www.mercadolibre.com.co/pastillas-shim...,buy_it_now,MCO,MCO371951,MCO-BICYCLE_DISC_BRAKE_PADS,http://http2.mlstatic.com/D_831848-MLU71764372...,COP,1,28160,32000.0,None,1,NaN,True,True,"{'store_pick_up': False, 'free_shipping': Fals...",2043-09-20T04:00:00.000Z,"{'id': 240348085, 'nickname': 'BIKETECH CARTAGO'}","[{'id': 'BRAND', 'name': 'Marca', 'value_id': ...","{'quantity': 36, 'amount': 782.22, 'rate': 0, ...",None,True,None,[],JBAL87326,NaN,NaN,NaN,NaN,MCO1930523048
1,Kit Bandas Elásticas Tubulares Set 11 Piezas J...,new,661800-MLU71875524679_092023,MCO27189100,gold_pro,https://www.mercadolibre.com.co/kit-bandas-ela...,buy_it_now,MCO,MCO179924,MCO-RESISTANCE_BANDS,http://http2.mlstatic.com/D_661800-MLU71875524...,COP,2,19339,NaN,None,500,NaN,True,True,"{'store_pick_up': False, 'free_shipping': Fals...",2044-02-03T04:00:00.000Z,"{'id': 263335107, 'nickname': 'VENTASBOGOTAOFI...","[{'id': 'BRAND', 'name': 'Marca', 'value_id': ...",None,None,True,None,[],None,NaN,NaN,NaN,NaN,MCO1389621843
2,Linterna Tactica Recargable De Larga Distancia...,new,710625-MLU73883611211_012024,MCO23772741,gold_special,https://www.mercadolibre.com.co/linterna-tacti...,buy_it_now,MCO,MCO47781,MCO-FLASHLIGHTS,http://http2.mlstatic.com/D_710625-MLU73883611...,COP,3,8344,NaN,None,50,NaN,True,True,"{'store_pick_up': False, 'free_shipping': Fals...",2043-08-27T04:00:00.000Z,"{'id': 443923880, 'nickname': 'ELECTRONICOSJAP...","[{'id': 'BRAND', 'name': 'Marca', 'value_id': ...",None,None,True,None,[],VRWO80836,NaN,NaN,NaN,NaN,MCO1333729713
3,Bomba De Mano Para Inflar Pedal 60 Psi Truper ...,new,912326-MLU72574318952_112023,MCO22609134,gold_special,https://www.mercadolibre.com.co/bomba-de-mano-...,buy_it_now,MCO,MCO371424,MCO-MANUAL_AND_FOOT_AIR_PUMPS,http://http2.mlstatic.com/D_912326-MLU72574318...,COP,4,18031,21990.0,None,1,1961.0,True,True,"{'store_pick_up': False, 'free_shipping': Fals...",2043-07-01T04:00:00.000Z,"{'id': 147458773, 'nickname': 'GMCOMUNICACIONE...","[{'id': 'BRAND', 'name': 'Marca', 'value_id': ...",None,None,True,None,[],OWAI66320,GM Comunicaciones,NaN,NaN,NaN,MCO1312014711
4,Colchón Inflable Doble Best Way Más 2 Almohada...,new,734619-MLU72651627087_112023,MCO23072789,gold_pro,https://www.mercadolibre.com.co/colchon-inflab...,buy_it_now,MCO,MCO47745,MCO-AIR_MATTRESSES,http://http2.mlstatic.com/D_734619-MLU72651627...,COP,5,92180,104750.0,None,1,NaN,True,True,"{'store_pick_up': False, 'free_shipping': True...",2044-02-22T13:34:22.000Z,"{'id': 549413071, 'nickname': 'SOLCOM_SOLUTION...","[{'id': 'BRAND', 'name': 'Marca', 'value_id': ...","{'quantity': 12, 'amount': 7681.67, 'rate': 0,...",None,True,None,[],JKRM22028,NaN,{'id': 33602181},NaN,NaN,MCO1563652150


In [79]:
df_items.columns

Index(['title', 'condition', 'thumbnail_id', 'catalog_product_id',
       'listing_type_id', 'permalink', 'buying_mode', 'site_id', 'category_id',
       'domain_id', 'thumbnail', 'currency_id', 'order_backend', 'price',
       'original_price', 'sale_price', 'available_quantity',
       'official_store_id', 'use_thumbnail_id', 'accepts_mercadopago',
       'shipping', 'stop_time', 'seller', 'attributes', 'installments',
       'winner_item_id', 'catalog_listing', 'discounts', 'promotions',
       'inventory_id', 'official_store_name', 'differential_pricing',
       'variation_filters', 'variations_data', 'item_id'],
      dtype='object')

# Creacion de columnas

In [80]:
#Informacion de Sellers
seller_info = json_normalize(df_items['seller'])
df_items = pd.concat([df_items, seller_info], axis=1)
df_items = df_items.drop('seller', axis=1)
df_items = df_items.rename(columns={'id': 'seller_id', 'nickname':'seller_nickname'})
#Informacion shipping
shipping_info = json_normalize(df_items['shipping'])
df_items = pd.concat([df_items, shipping_info], axis=1)
df_items = df_items.drop('shipping', axis=1)
#Informacion installments
installments = json_normalize(df_items['installments'])
df_items = pd.concat([df_items, installments], axis=1)
df_items = df_items.drop('installments', axis=1)

In [81]:
df_items.head()

,title,condition,thumbnail_id,catalog_product_id,listing_type_id,permalink,buying_mode,site_id,category_id,domain_id,thumbnail,currency_id,order_backend,price,original_price,sale_price,available_quantity,official_store_id,use_thumbnail_id,accepts_mercadopago,stop_time,attributes,winner_item_id,catalog_listing,discounts,promotions,inventory_id,official_store_name,differential_pricing,variation_filters,variations_data,item_id,seller_id,seller_nickname,store_pick_up,free_shipping,logistic_type,mode,tags,benefits,promise,quantity,amount,rate,currency_id
0,Pastillas Shimano B05s Originales 50% Más Resi...,new,831848-MLU71764372657_092023,MCO27114730,gold_special,https://www.mercadolibre.com.co/pastillas-shim...,buy_it_now,MCO,MCO371951,MCO-BICYCLE_DISC_BRAKE_PADS,http://http2.mlstatic.com/D_831848-MLU71764372...,COP,1,28160,32000.0,None,1,NaN,True,True,2043-09-20T04:00:00.000Z,"[{'id': 'BRAND', 'name': 'Marca', 'value_id': ...",None,True,None,[],JBAL87326,NaN,NaN,NaN,NaN,MCO1930523048,240348085,BIKETECH CARTAGO,False,False,cross_docking,me2,[],None,None,36.0,782.22,0.0,COP
1,Kit Bandas Elásticas Tubulares Set 11 Piezas J...,new,661800-MLU71875524679_092023,MCO27189100,gold_pro,https://www.mercadolibre.com.co/kit-bandas-ela...,buy_it_now,MCO,MCO179924,MCO-RESISTANCE_BANDS,http://http2.mlstatic.com/D_661800-MLU71875524...,COP,2,19339,NaN,None,500,NaN,True,True,2044-02-03T04:00:00.000Z,"[{'id': 'BRAND', 'name': 'Marca', 'value_id': ...",None,True,None,[],None,NaN,NaN,NaN,NaN,MCO1389621843,263335107,VENTASBOGOTAOFICIAL,False,False,cross_docking,me2,[self_service_in],None,None,NaN,NaN,NaN,NaN
2,Linterna Tactica Recargable De Larga Distancia...,new,710625-MLU73883611211_012024,MCO23772741,gold_special,https://www.mercadolibre.com.co/linterna-tacti...,buy_it_now,MCO,MCO47781,MCO-FLASHLIGHTS,http://http2.mlstatic.com/D_710625-MLU73883611...,COP,3,8344,NaN,None,50,NaN,True,True,2043-08-27T04:00:00.000Z,"[{'id': 'BRAND', 'name': 'Marca', 'value_id': ...",None,True,None,[],VRWO80836,NaN,NaN,NaN,NaN,MCO1333729713,443923880,ELECTRONICOSJAPONP,False,False,drop_off,me2,[],None,None,NaN,NaN,NaN,NaN
3,Bomba De Mano Para Inflar Pedal 60 Psi Truper ...,new,912326-MLU72574318952_112023,MCO22609134,gold_special,https://www.mercadolibre.com.co/bomba-de-mano-...,buy_it_now,MCO,MCO371424,MCO-MANUAL_AND_FOOT_AIR_PUMPS,http://http2.mlstatic.com/D_912326-MLU72574318...,COP,4,18031,21990.0,None,1,1961.0,True,True,2043-07-01T04:00:00.000Z,"[{'id': 'BRAND', 'name': 'Marca', 'value_id': ...",None,True,None,[],OWAI66320,GM Comunicaciones,NaN,NaN,NaN,MCO1312014711,147458773,GMCOMUNICACIONESY ELECTRONICA,False,False,fulfillment,me2,"[fulfillment, self_service_in]",None,None,NaN,NaN,NaN,NaN
4,Colchón Inflable Doble Best Way Más 2 Almohada...,new,734619-MLU72651627087_112023,MCO23072789,gold_pro,https://www.mercadolibre.com.co/colchon-inflab...,buy_it_now,MCO,MCO47745,MCO-AIR_MATTRESSES,http://http2.mlstatic.com/D_734619-MLU72651627...,COP,5,92180,104750.0,None,1,NaN,True,True,2044-02-22T13:34:22.000Z,"[{'id': 'BRAND', 'name': 'Marca', 'value_id': ...",None,True,None,[],JKRM22028,NaN,{'id': 33602181},NaN,NaN,MCO1563652150,549413071,SOLCOM_SOLUTIONS_ECOMMERCE,False,True,cross_docking,me2,[mandatory_free_shipping],None,None,12.0,7681.67,0.0,COP


In [82]:
df_items.columns

Index(['title', 'condition', 'thumbnail_id', 'catalog_product_id',
       'listing_type_id', 'permalink', 'buying_mode', 'site_id', 'category_id',
       'domain_id', 'thumbnail', 'currency_id', 'order_backend', 'price',
       'original_price', 'sale_price', 'available_quantity',
       'official_store_id', 'use_thumbnail_id', 'accepts_mercadopago',
       'stop_time', 'attributes', 'winner_item_id', 'catalog_listing',
       'discounts', 'promotions', 'inventory_id', 'official_store_name',
       'differential_pricing', 'variation_filters', 'variations_data',
       'item_id', 'seller_id', 'seller_nickname', 'store_pick_up',
       'free_shipping', 'logistic_type', 'mode', 'tags', 'benefits', 'promise',
       'quantity', 'amount', 'rate', 'currency_id'],
      dtype='object')

# Exploracion Datos

In [85]:
total_sales = df_items.groupby(['seller_id', 'seller_nickname'])['quantity'].sum().reset_index()
# Ordena y visualiza los sellers con las mayores ventas totales
top_sellers = total_sales.sort_values(by='quantity', ascending=False)
top_sellers.head(15)

,seller_id,seller_nickname,quantity
137,240348085,BIKETECH CARTAGO,8580.0
83,160820235,BUILDER CO,6336.0
80,155147039,WGDISTRIBUCIONDEREPUESTOS,4500.0
24,53544979,VOY ENBICI,4392.0
99,178840084,KORINTIO.COM,4176.0
143,247730346,AKTIVABRANDS COLOMBIA,3852.0
156,271509738,HELLOFTHENORTHSTORE,3708.0
60,118807519,HOMESALE.COM.CO,3708.0
125,212962423,WWW.BIG44SALE.COM,2796.0
183,369346540,WWW.HOMESALE.COM.CO,2628.0
